In [17]:
import { zodToJsonSchema } from "zod-to-json-schema";
import { z } from "zod";
import { ChatOpenAI } from "@langchain/openai"
import { Ollama } from "@langchain/community/llms/ollama";
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools"
import { RunnableSequence, RunnableBranch, RunnablePassthrough } from "@langchain/core/runnables"
import { ChatPromptTemplate, PromptTemplate } from "@langchain/core/prompts"
const classifySchema = z.object({
    type: z.enum(["科普", "编程", "一般问题"]).describe("用户提问的分类")
})

const model = new ChatOpenAI({
    configuration: {
            baseURL: "https://api.aigc369.com/v1",
        },
    temperature: 0 
})

const modelWithTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "classifyQuestion",
                description: "对用户的提问进行分类",
                parameters: zodToJsonSchema(classifySchema),
            }
        }
    ],
    tool_choice: {
        type: "function",
        function: {
           name: "classifyQuestion"
        }
    }
})

const prompt = ChatPromptTemplate.fromMessages([
    ["system", `仔细思考，你有充足的时间进行严谨的思考，然后对用户的问题进行分类，
    当你无法分类到特定分类时，可以分类到 "一般问题"`],
    ["human", "{input}"]
])

const classifyChain = RunnableSequence.from([
    prompt,
    modelWithTools,
    new JsonOutputToolsParser(),
    (input) => {
        const type = input[0]?.args?.type
        return type ? type : "一般问题"
    }
])

await classifyChain.invoke({
    "input": "鲸鱼是哺乳动物么？"
})

TypeError: Cannot read properties of undefined (reading 'additional_kwargs')

In [12]:
import { StringOutputParser } from "@langchain/core/output_parsers";
const answeringModel = new Ollama({
    baseUrl: "http://localhost:11434", 
    model: "llama3", 
    temperature: 0.7
})
const sciencePrompt = PromptTemplate.fromTemplate(
    `作为一位科普专家，你需要解答以下问题，尽可能提供详细、准确和易于理解的答案：
    问题：{input}
    答案：`
)
const programmingPrompt = PromptTemplate.fromTemplate(
    `作为一位编程专家，你需要解答以下编程相关的问题，尽可能提供详细、准确和实用的答案：
    问题：{input}
    答案：`
)
const generalPrompt = PromptTemplate.fromTemplate(
    `请回答以下一般性问题，尽可能提供全面和有深度的答案：
    问题：{input}
    答案：`
)
const scienceChain = RunnableSequence.from([
    sciencePrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "科普专家"
    }
])

const programmingChain = RunnableSequence.from([
    programmingPrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "编程大师"
    }
])

const generalChain = RunnableSequence.from([
    generalPrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "通识专家"
    }
    
])

const branch = RunnableBranch.from([
    [
        (input => input.type.includes("科普")),
        scienceChain,
    ],
    [
        (input => input.type.includes("编程")),
        programmingChain
    ],
    generalChain
])

In [14]:
const outputTemplate = PromptTemplate.fromTemplate(
    `感谢您的提问，这是来自{role}的专业回答:
    {output}
    `
)

const finalChain = RunnableSequence.from([
    {
        type: classifyChain,
        input: input => input.input
    },
    branch,
    (input) => outputTemplate.format(input)
])
const res = await finalChain.invoke({
    "input": "鲸鱼是哺乳动物么？"
})

console.log(res)


感谢您的提问，这是来自科普专家的专业回答:
    As a science communicator, I'd be happy to help clarify this question! 🐳🦇

Answer: Yes, whales are mammals, and specifically, they belong to the order Cetacea. In fact, they are the largest group of mammals that live in the water.

Here's why:

1. **Warm-bloodedness**: Like all mammals, whales are endothermic, meaning they generate heat internally to maintain a stable body temperature, regardless of their surroundings.
2. **Breathing air**: Whales surface to breathe air through their blowholes (modified nostrils) and have lungs adapted for gas exchange. They don't use gills like fish do.
3. **Mammary glands**: Female whales produce milk to feed their young, just like all other mammals. This is an essential characteristic that defines mammals.
4. **Three middle ear bones**: Whales, like all mammals, have three middle ear bones (ossicles) that transmit sound vibrations to the inner ear.
5. **Brain structure**: Whales possess a large brain-to-body mass ratio, whi